In [48]:
import numpy as np
import pandas as pd
import torch

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [50]:
df = pd.read_csv('./data/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [51]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

In [53]:
torch.manual_seed(13)

In [54]:
x_train_scaled = x_train / 255.0
x_test_scaled = x_test / 255.0

In [55]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        super().__init__()
        self.features = torch.tensor(features, dtype = torch.float32).reshape(-1, 1, 28, 28)
        self.labels = torch.tensor(labels, dtype = torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [56]:
trainds = CustomDataset(x_train_scaled, y_train)
testds = CustomDataset(x_test_scaled, y_test)

In [57]:
train_loader = DataLoader(trainds, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(testds, batch_size=32, shuffle=False, pin_memory=True)

In [65]:
import torch.nn as nn
from torch import optim

class MyCNN(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32,  64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(64, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = self.classifier(out)
        return out
        

In [66]:
learning_rate = 0.01
epochs = 100

In [67]:
model = MyCNN(1)
model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr= learning_rate)

In [68]:
for epoch in range(epochs):
    total_epoch_loss = 0
    
    for bfeatures, blabels in train_loader:
        bfeatures, blabels = bfeatures.to(device), blabels.to(device)

        out = model(bfeatures)
        loss = loss_function(out, blabels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        total_epoch_loss += loss.item()

    avg_loss = total_epoch_loss / len(train_loader)
    print(f'Epoch: {epoch + 1} | loss: {avg_loss}')

Epoch: 1 | loss: 1.4049393924077351
Epoch: 2 | loss: 0.8173552016417186
Epoch: 3 | loss: 0.6894902578989665
Epoch: 4 | loss: 0.587620564699173
Epoch: 5 | loss: 0.5334065698583921
Epoch: 6 | loss: 0.4717119687795639
Epoch: 7 | loss: 0.44318660348653793
Epoch: 8 | loss: 0.4036156079173088
Epoch: 9 | loss: 0.365979292144378
Epoch: 10 | loss: 0.34461126625537875
Epoch: 11 | loss: 0.31392450322707494
Epoch: 12 | loss: 0.29746637677152954
Epoch: 13 | loss: 0.26761982768774034
Epoch: 14 | loss: 0.26275569359461465
Epoch: 15 | loss: 0.23012544895211856
Epoch: 16 | loss: 0.22116103939712048
Epoch: 17 | loss: 0.1883732609078288
Epoch: 18 | loss: 0.17880475054184597
Epoch: 19 | loss: 0.16702851239591837
Epoch: 20 | loss: 0.1493351513147354
Epoch: 21 | loss: 0.14190150102600455
Epoch: 22 | loss: 0.13728212965031464
Epoch: 23 | loss: 0.1234069828564922
Epoch: 24 | loss: 0.1152123107202351
Epoch: 25 | loss: 0.10217290921136737
Epoch: 26 | loss: 0.09108009027317167
Epoch: 27 | loss: 0.079781370402003

In [69]:
model.eval()

MyCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [70]:
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8875


In [71]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9997916666666666
